# one-hot Encoding

문자를 숫자로 바꾸는 기법 들 중 단어를 표현하는 가장 기본적인 표현 방법

## 단어 집합(vocabulary)

서로 다른 단어들의 집합(book과 books는 서로 다른 단어)\
**집합이므로 중복은 허용하지 않는다**
text의 모든 단어를 중복을 허용하지 않고 모아놓으면 단어 집합이 된다.\
단어 집합에 고유한 정수를 부여하는 integer encoding을 진행한다.

ex) text에 단어가 총 5000개가 존재한다면, 단어 집합의 크기는 5,000\
5000개의 단어가 있는 이 단어 집합의 단어들마다 1번부터 5000번까지 index를 부여한다고 하자.\
book는 150번, dog는 171번, love는 192번, books는 212번

이렇게 각 단어에 고유한 integer index를 부여했다 하자.\
이 숫자로 바뀐 단어들을 vector로 다루게 될 것



## One-Hot Encoding

단어 집합(vocabulary)의 크기를 vector의 차원으로 하고, 표현하고 싶은 단어의 index에 1의 값을 부여\
다른 index에는 0을 부여하는 단어의 vector 표현 방식

다음의 두 가지 과정을 거친다

**1) integer encoding을 수행한다. 즉 각 단어에 고유한 정수를 부여한다**\
**2) 표현하고 싶은 단어의 고유한 정수를 index로 간주, 해당 위치에 1을 부여, 다른 단어의 index의 위치에는 0을 부여**


**문장: 나는 자연어 처리를 배운다**

Okt 형태소 분석기를 통해 문장에 대해서 tokenizing을 수행

In [2]:
from konlpy.tag import Okt

okt = Okt()
tokens = okt.morphs("나는 자연어 처리를 배운다")
print(tokens)

['나', '는', '자연어', '처리', '를', '배운다']


각 토큰에 고유한 정수를 부여\
문장이 짧은 경우 각 단어의 빈도수를 고려하지 않지만, 빈도수 순으로 단어를 정렬하여 정수를 부여하는 경우가 많다

In [4]:
word_to_index = {word:index for index, word in enumerate(tokens)}
print('단어 집합 : ', word_to_index)

단어 집합 :  {'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


토큰을 입력하면 해당 토큰에 대한 one-hot vector를 만들어내는 함수를 만들었다

In [5]:
def one_hot_encoding(word,word_to_index):
    one_hot_vector = [0]*(len(word_to_index))
    index = word_to_index[word]
    one_hot_vector[index] = 1
    return one_hot_vector
    

'자연어'라는 단어의 one-hot vector를 얻어보자

In [6]:
one_hot_encoding("자연어", word_to_index)

[0, 0, 1, 0, 0, 0]

'자연어'는 정수 2 이므로 one-hot vector는 index 2의 값이 1이며, 나머지 값은 0인 vector가 나온다.

## Keras를 이용한 one-hot encoding

위에서는 one-hot encoding을 이해하기 위해 python으로 직접 코드를 작성\
keras는 one-hot encoding을 이용하기 위한 도구로 **to_categorical()** 을 제공\
keras 만으로 정수 인코딩과 원-핫 encoding을 순차적으로 해보자

In [7]:
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

위와 같은 문장을 keras tokenizer를 이용한 integer encoding은 다음과 같다

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('단어 집합 :', tokenizer.word_index)

단어 집합 : {'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


생성된 단어 집합 (vocabularly)에 있는 단어들로만 구성된 text가 있다면\
text_to_sequences()를 통해 이를 integer sequence로 변환이 가능하다\
생성된 단어 집합 내의 일부 단어들로만 구성된 sub text인 sub_text를 만들어 확인해보자

In [9]:
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"  # 단어 집합이므로 중복이 존재하지 x

encoded = tokenizer.texts_to_sequences([sub_text])[0]
print(encoded)

[2, 5, 1, 6, 3, 7]


지금까지는 integer encoding 하면서 배웠던 것들\
이 결과를 이용하여 one-hot encoding을 진행할 것\
keras는 integer encoding된 결과로부터 one-hot encoding을 수행하는 to_categorical()을 지원

In [10]:
one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


[[0. 0. 1. 0. 0. 0. 0. 0.]  # index 2의 one-hot vector\
 [0. 0. 0. 0. 0. 1. 0. 0.]  # index 5의 one-hot vector\
 [0. 1. 0. 0. 0. 0. 0. 0.]  # index 1의 one-hot vector\
 [0. 0. 0. 0. 0. 0. 1. 0.]  # index 6의 one-hot vector\
 [0. 0. 0. 1. 0. 0. 0. 0.]  # index 3의 one-hot vector\
 [0. 0. 0. 0. 0. 0. 0. 1.]] # index 7의 one-hot vector

 이 결과는 "점심 먹으러 갈래 메뉴는 햄버거 최고야"라는 문장이 [2,5,1,6,3,7]로 integer encoding 되고\
 각각이 encoding된 결과를 index로 one-hot encoding된 결과
